In [18]:
import requests
from bs4 import BeautifulSoup
import time
import os
import openai

folder_path = "Unsheathed"
# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key

GPT_MODEL = "gpt-3.5-turbo"

conversation = [
    { "role" : "system", 
      "content" : "Translate to English. This is a Xianxia novel. No additional text.\
                  Preserve the tone, narrative voice, and names of people as provided. \
                  Names of places, organizations, techniquees can generally be translated to a nice sounding name which retains the original idea.\
                  Idioms and metaphors should be translated literally, followed by a short explanation if needed. \
                  Cultural terms can be left untranslated unless translation improves readability.\
                  Make sure honorifics are placed wherever required.\
                  Do not cut sentences or replace lists with \'etc\'.\
                  Edit the sentences and dialogue to flow smoothly in English."},
  ]

# conversation_nopop = [
#     {"role" : "system", "content" : "Translate this from Chinese to English. The context being that this is a Xianxia novel. Do not write anything else"},
#   ]

In [19]:
def GPTranslate(para, totaltokens):
    conversation.append({"role": "user", "content": para})
        
    response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages= conversation
    )

    conversation.pop()

    print("for para of length", len(para), "\n", response['usage'])
    totaltokens += response['usage']['total_tokens']
    # print(response['choices'][0]['message']['content'])
    
    return response['choices'][0]['message']['content'], totaltokens

In [20]:
url = 'https://lnmtl.com/chapter/come-sword-book-1-chapter-1'
Chapters = 1

In [ ]:
# experimenting with higher thresholds
        # tokens for chapter 1, 2, 3 on threshold = 500  -> 7031,7512,7001 (time 5m something~) (never cut)
        # tokens for chapter 1, 2, 3 on threshold = 1800 -> 6719,7216,6611 (time 5m 20s~) (maybe cut, dont remember)
        # tokens for chapter 1, 2, 3 on threshold = 2500 -> Almost always cuts out the translation
        # tokens for chapter 1, 2, 3 on threshold = 1650 -> 6570,7269,6609 (time 5m 25s) (never cut) ~4% saving 500 threshold
        # tokens for chapter 1, 2, 3 : 1650, new instrcn -> 6869,7434,6821 (time 5m 30s) (never cut) ~2% more than 1650 raw
        # lost track of the time counter, I'm willing to say its faster on the higher threshold.
        # Max tokens is 4k, and visually looking at sentence length and input token from response gives me a ~1:1.35 feel
        # whereas sentence length to output token from response is ~1:0.9 feel, so optimal input length ~ 1.8k
        # to be safe on fluctuations, 1:1.4 + 1:1, 1.7k. 1650 feels fine? can push 1700
        # max ratios reached 1:1.4, 1:.92 (raw instr)
        # max ratios reached 1:1.44, 1:0.87 (new instr)
        
        # <input para len - prompt tokens - ratio to para len, completion tokens - ratio to para len>
        # threshold 2500 (NA result) (max token reached 4096)
        # 2408 - 3347 - 1.39, NA
        # 619  - 800 - 1.3, 542 - .87
        # 2433 - 3302 - 1.36, NA
        # 836  - 1131 - 1.35, 735 - .87
        # 2392 - 3158 - 1.32, NA
        # 637  - 852 - 1.33, 588 - .92

        # threshold 1650 (all produce output) (max token reached 3720) (10% below limit)
        # 1629 - 2290 - 1.4, 1289 - .8
        # 1398 - 1857 - 1.3, 1101 - .78
        # 1638 - 2263 - 1.38, 1457 - .88
        # 1631 - 2170 - 1.33, 1348 - .82
        # 1567 - 2086 - 1.33, 1304 - .83
        # 1462 - 1925 - 1.31, 1265 - 0.86
        
        # threshold 1650, new instructions (max tokens is 3756, 3700+ twice) (can get close to limit i feel)
        # 1629 - 2346 - 1.44, 1410 - .86
        # 1398 - 1913 - 1.36, 1109 - .79
        # 1638 - 2319 - 1.41, 1434 - .87
        # 1631 - 2226 - 1.36, 1368 - .83
        # 1567 - 2142 - 1.36, 1333 - .85
        # 1462 - 1981 - 1.35, 1278 - .87

        # From observations, lower thresholds can lead to worse translations simply because 
        # Chinese requires larger context for things like tense, gender, etc
        # Virtually no diff in transation quality that I can see between 1650&1800.
        # Can see between all translations, the issue that some words get left untranslated sometimes and translated others
        # This is commonplace, but issue is consistency can be improved upon. Should experiment with instructions
        
        # Original instruction
        #  {"role" : "system", 
        #  "content" : "Translate to English. Xianxia novel context. No additional text.}
        
        
        # New instruction
        #     {"role" : "system", 
        #  "content" : "Translate to English. Xianxia novel context. No additional text. \
        #               Preserve the tone, narrative voice, and names of people as provided. Names of places or organizations can generally be translated literally.\
        #               Idioms and metaphors should to placed literally, followed by a short explanation if needed. Cultural terms can be left untranslated unless translation improves readability."}
        
        
        # New New Instruction - {"role" : "system", 
        #  "content" : "Translate to English. This is a Xianxia novel. No additional text.\
        #               Preserve the tone, narrative voice, and names of people as provided. \
        #               Names of places, organizations, techniquees can generally be translated to a nice sounding name which retains the original idea.\
        #               Idioms and metaphors should be translated literally, followed by a short explanation if needed. \
        #               Cultural terms can be left untranslated unless translation improves readability.\
        #               Make sure honorifics are placed wherever required.\
        #               Do not cut sentences or replace lists with \'etc\'.\
        #               Edit the sentences and dialogue to flow smoothly in English."}

In [21]:
for i in range(3):
    
    print("at chapter", url)
    
    # time.sleep(3)
    response = requests.get(url)

    paras = []

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all elements with the specified class 'original'
        nextchap = soup.find('li', class_='next').find('a')['href']
        # print(nextchap)
        url = nextchap
        
        # Find all elements with the specified class 'original'
        original_sentences = soup.find_all('sentence', class_='original')
        
        # Extract and print the text from each element
        for sentence in original_sentences:
            line = sentence.get_text()
            paras.append(line)
        
                
        # Initialize variables
        running_length = 0  # Start with 0
        threshold = 1650  # Set your desired threshold here
        
        result = []  # Store the joined paragraphs
        totaltokens = 0 # Total tokens used in translating one chapter
        translation = [] # Storing the translation

        title, totaltokens = GPTranslate(soup.find('span', class_="chapter-title")['data-content'], totaltokens)
        # title = "Chapter " + str(Chapters) + " - " + title.rstrip(".") + " (" + str(threshold) + ")"
        # title = "Chapter " + str(Chapters) + " - " + title.rstrip(".") + " (" + str(threshold) + ", new inst)"
        title = "Chapter " + str(Chapters) + " - " + title.rstrip(".") + " (" + str(threshold) + ", new new inst)"
        print(title)
        Chapters += 1
        
        # Iterate over the paras
        for para in paras:
            # Check if adding the current paragraph exceeds the threshold
            if running_length + len(para) <= threshold:
                running_length += len(para) # Update the running length
                result.append(para)
                
            else:
                # print(''.join(result))
                # time.sleep(2)
                translated_result, totaltokens = GPTranslate(''.join(result), totaltokens)
                print(translated_result)
                translation.append(translated_result)
                
                # Reset the result and running_length
                result = [para]  # Start a new result with the current paragraph
                running_length = len(para)  # Set running_length to the current paragraph's length

        # print(''.join(result))
        # time.sleep(2)
        translated_result, totaltokens = GPTranslate(''.join(result), totaltokens)
        print(translated_result)
        translation.append(translated_result)
        
        tokensusedline = "Note: This translation used " + str(totaltokens) + " GPT tokens"
        translation.append(tokensusedline)
        print(tokensusedline)
        
        filename = f"{title}.txt"
        full_path = os.path.join(folder_path, filename)
        with open(full_path, 'w') as file:
            file.write(''.join(translation))
        print("saved to", full_path)
            
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

at chapter https://lnmtl.com/chapter/come-sword-book-1-chapter-1
for para of length 2 
 {
  "prompt_tokens": 86,
  "completion_tokens": 5,
  "total_tokens": 91
}
Chapter 1 - Awakening of Insects (1650, new inst)
for para of length 1629 
 {
  "prompt_tokens": 2346,
  "completion_tokens": 1410,
  "total_tokens": 3756
}
On the second day of the second month, the dragon awakens.

In the twilight, in a quiet corner of a town called Niping Lane, there was a lonely and skinny young man. According to tradition, he held a candle in one hand and a peach branch in the other, illuminating the beams, walls, and wooden bed, using the peach branch to knock and drive away snakes, scorpions, centipedes, and so on. He murmured the old saying that had been passed down through generations in this town: "On the second day of the second month, candles illuminate the beams, peaches hit the walls, and there is nowhere for snakes and worms to hide in the human world."

The young man's surname was Chen, and his

In [22]:
# def GPTranslate_nopop(para):
#     conversation_nopop.append({"role": "user", "content": para})
        
#     response = openai.ChatCompletion.create(
#     model=GPT_MODEL,
#     messages= conversation_nopop
#     )
    
#     conversation_nopop.append({"role" : "assistant", "content": response['choices'][0]['message']['content']})

#     # conversation.pop()

#     # print("for para of length", len(para), "\n", response['usage'])
#     tokensused = response['usage']['total_tokens']
#     # print(response['choices'][0]['message']['content'])
    
#     return response['choices'][0]['message']['content'], tokensused

# url = 'https://lnmtl.com/chapter/come-sword-book-1-chapter-1'
# Chapters = 1
# for i in range(1):
#     conversation_nopop = conversation_nopop[:1]
#     print("at chapter", url)
    
#     # time.sleep(3)
#     response = requests.get(url)

#     paras = []

#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Parse the HTML content of the page using BeautifulSoup
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Find all elements with the specified class 'original'
#         nextchap = soup.find('li', class_='next').find('a')['href']
#         # print(nextchap)
#         url = nextchap
        
#         # Find all elements with the specified class 'original'
#         original_sentences = soup.find_all('sentence', class_='original')
        
#         # Extract and print the text from each element
#         for sentence in original_sentences:
#             line = sentence.get_text()
#             paras.append(line)
        
                
#         # Initialize variables
#         running_length = 0  # Start with 0
#         threshold = 500  # Set your desired threshold here
#         result = []  # Store the joined paragraphs
#         runningtokens = 0 # Total tokens used in current conversation
#         totaltokens = 0 # Total tokens used overall
#         translation = [] # Storing the translation

        
#         title, runningtokens = GPTranslate_nopop(soup.find('span', class_="chapter-title")['data-content'])
#         totaltokens += runningtokens
#         title = "NPChapter " + str(Chapters) + " - " + title.rstrip(".")
#         print(title)
#         Chapters += 1
        
#         # Iterate over the paras
#         for para in paras:
#             # Check if adding the current paragraph exceeds the threshold
#             if running_length + len(para) <= threshold:
#                 running_length += len(para) # Update the running length
#                 result.append(para)
                
#             else:
#                 print("entered else block")
#                 # print(''.join(result))
#                 # time.sleep(2)
#                 print("conversation no pop \n", conversation_nopop[-1])
#                 translated_result, runningtokens = GPTranslate_nopop(''.join(result))
#                 print("translation successful, with running", runningtokens)
#                 totaltokens += runningtokens

#                 # print(translated_result)
#                 translation.append(translated_result)
                
#                 if runningtokens >= 3800:
#                     print("pruning tokenusage")
#                     conversation_nopop = [conversation_nopop[0], conversation_nopop[-1]]
                
#                 # Reset the result and running_length
#                 result = [para]  # Start a new result with the current paragraph
#                 running_length = len(para)  # Set running_length to the current paragraph's length

#         # print(''.join(result))
#         # time.sleep(2)
#         translated_result, runningtokens = GPTranslate_nopop(''.join(result))
#         # print(translated_result)
#         translation.append(translated_result)
        
#         tokensusedline = "Note: This translation used " + str(totaltokens) + " GPT tokens"
#         translation.append(tokensusedline)
#         print(tokensusedline)
        
#         filename = f"{title}.txt"
#         full_path = os.path.join(folder_path, filename)
#         with open(full_path, 'w') as file:
#             file.write(''.join(translation))
#         print("saved to", full_path)
            
#     else:
#         print("Failed to retrieve the webpage. Status code:", response.status_code)